In [3]:
import langchain

In [ ]:
import dotenv
import os
dotenv.load_dotenv("../.env")

: 

In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import TavilySearchResults
from langchain_tavily import TavilySearch
from langchain.prompts import ChatPromptTemplate
import os

# === 1. Initialize the Gemini-based LLM ===
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # or "gemini-2.5-flash" for lower latency & cost
    temperature=0.0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)



# 2. Set up search tool (requires TAVILY_API_KEY in your env)
search = TavilySearch(k=5, tavily_api_key=os.environ["TAVILY_API_KEY"])

# 3. Define prompt
prompt_template = ChatPromptTemplate.from_template("""
You are an AI business analyst.  
Given the following raw web search results about the company "{company}",  
summarize important details for a potential business partner.  
Highlight:
- Core business activities  
- Financial health (if mentioned)  
- Reputation and recent news  
- Any red flags (lawsuits, fraud, layoffs, controversies)  

Results:
{results}

Return a concise, bullet-point summary.
""")

# 4. Function to analyze a company
def analyze_company(company_name):
    search_results = search.run(company_name + " company news financial red flags")
    prompt = prompt_template.format(company=company_name, results=search_results)
    summary = llm.invoke(prompt)
    return summary.content

# Example usage
if __name__ == "__main__":
    print(analyze_company("WeWork"))

Here's a summary of important details about WeWork for a potential business partner:

*   **Core Business Activities:** WeWork primarily provides co-working spaces.
*   **Financial Health:**
    *   Considered a major red flag due to its financial health.
    *   In 2018, despite $1.8 billion in revenue, costs were exceptionally high.
    *   Characterized by increasing leverage and an inflated valuation.
*   **Reputation and Recent News:**
    *   Widely criticized and viewed as a "cautionary tale" in the startup world.
    *   Its business model has been questioned for a lack of innovation.
    *   Associated with significant controversy, including a "cult" narrative around its former leadership.
*   **Red Flags:**
    *   **Severe Financial Concerns:** High costs, increasing debt, and an overvalued assessment.
    *   **Questionable Business Model:** Perceived lack of innovation.
    *   **Insider Selling:** Indicated by reports ahead of its IPO.
    *   **Strong Warnings:** Financi

In [7]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import TavilySearchResults
import os

# --- 2. Define state ---
class CompanyState(TypedDict):
    company: str
    search_results: str
    summary: str
    red_flags: str
    report: str

# --- 3. Initialize LLM & search ---
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

search_tool = TavilySearchResults(k=5)

# --- 4. Prompts ---
summarize_prompt = ChatPromptTemplate.from_template("""
You are a business intelligence analyst.
Summarize search results for "{company}" covering:
- Core business activities
- Financial health
- Recent news or events
Results:
{results}
""")

redflag_prompt = ChatPromptTemplate.from_template("""
From this summary, identify any business risks:
- Lawsuits
- Fraud
- Layoffs
- Bankruptcy
If none found, say: "No significant red flags identified."
Summary:
{summary}
""")

# --- 5. Nodes ---
def search_node(state: CompanyState) -> CompanyState:
    results = search_tool.run(f"{state['company']} news financial lawsuits controversies")
    return {**state, "search_results": results}

def summarize_node(state: CompanyState) -> CompanyState:
    prompt = summarize_prompt.format(company=state["company"], results=state["search_results"])
    res = llm.invoke(prompt)
    return {**state, "summary": res.content}

def redflag_node(state: CompanyState) -> CompanyState:
    prompt = redflag_prompt.format(summary=state["summary"])
    res = llm.invoke(prompt)
    return {**state, "red_flags": res.content}

def report_node(state: CompanyState) -> CompanyState:
    report = (
        f"--- Company Intelligence Report ---\n\n"
        f"Company: {state['company']}\n\n"
        f"Summary:\n{state['summary']}\n\n"
        f"Red Flags:\n{state['red_flags']}"
    )
    return {**state, "report": report}

# --- 6. Build graph ---
workflow = StateGraph(CompanyState)

workflow.add_node("search", search_node)
workflow.add_node("summarize", summarize_node)
workflow.add_node("red_flags", redflag_node)
workflow.add_node("report", report_node)

workflow.add_edge(START, "search")
workflow.add_edge("search", "summarize")
workflow.add_edge("summarize", "red_flags")
workflow.add_edge("red_flags", "report")
workflow.add_edge("report", END)

app = workflow.compile()

# --- 7. Run ---
if __name__ == "__main__":
    result = app.invoke({"company": "WeWork"})
    print(result["report"])


--- Company Intelligence Report ---

Company: WeWork

Summary:
Okay, here's a summary of WeWork based on the provided search results, covering its core business, financial health, and recent news:

**Core Business Activities:**

*   WeWork's core business revolves around leasing office buildings and transforming them into shared workspaces. They then rent out this space to freelancers, startups, and larger corporations, often on flexible, short-term leases. The company provides amenities like free beer to tenants.

**Financial Health:**

*   **Pre-Bankruptcy Issues:** WeWork's business model was known to be expensive with a questionable path to profitability. The company was significantly overextended in terms of building leases and renovation costs.
*   **2019 IPO Failure:** WeWork's attempt to go public in 2019 failed spectacularly due to concerns about its financials, business practices, and the leadership of its CEO, Adam Neumann. This led to major funders pulling out and the compa

In [8]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
from langchain_tavily import TavilySearch
from langgraph.prebuilt import create_react_agent

# Initialize Tavily Search Tool
tavily_search_tool = TavilySearch(
    max_results=5,
)

agent = create_react_agent(llm, [tavily_search_tool])

user_input = "What are potential red flag about company called OpenAi as a potential business partner"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What are potential red flag about company called OpenAi as a potential business partner
================================== Ai Message ==================================
Tool Calls:
  tavily_search (4efe7d50-76d7-45d1-af95-0ec9e562101a)
 Call ID: 4efe7d50-76d7-45d1-af95-0ec9e562101a
  Args:
    query: OpenAI red flags business partner OR concerns OpenAI partnership
================================= Tool Message =================================
Name: tavily_search

{"query": "OpenAI red flags business partner OR concerns OpenAI partnership", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.linkedin.com/posts/allisonlcarter_the-scoop-deepseek-raises-red-flags-for-activity-7290401488625942528-sD6i", "title": "DeepSeek raises red flags for OpenAI, U.S. Navy - PR Daily | Allison ...", "content": "I caution anyone using this for their business not to share any propriet

In [1]:
import gradio as gr
import dotenv


In [2]:
dotenv.load_dotenv(".env")

False

In [3]:
from langchain_tavily import TavilySearch

tavily_search_tool = TavilySearch(
    max_results=5,
    topic="general",
)

In [33]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
agent = create_react_agent(
    tools=[tavily_search_tool],
    model=llm
)

In [ ]:
def ask_agent_stream(company_name):
    user_input = f"What are potential red flag about company called {company_name} as a potential business partner. Prepare a markdown summary. Provide links and informations you got from tools"

    response = agent.invoke({"messages": user_input})
    return response["messages"][-1].content




In [ ]:
import gradio as gr

with gr.Blocks() as app:
    gr.Markdown("## Company Application Processor")

    company_name = gr.Textbox(label="Company name")
    start_button = gr.Button("Start information gathering")

    summary = gr.Markdown("**Summary**")
    result = gr.Markdown("", container=True)

    start_button.click(fn=ask_agent_stream, inputs=[company_name], outputs=[result])

app.launch()


* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


In [ ]:
['__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__ior__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__ror__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']


In [ ]:
{'messages': [HumanMessage(content='What are potential red flag about company called Spotify as a potential business partner', additional_kwargs={}, response_metadata={}, id='a9d44795-fbe3-4596-912e-5c1d66d5b590'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "Spotify potential red flags as business partner"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--14317fbe-6659-45af-ae69-b48dadc43bc7-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'Spotify potential red flags as business partner'}, 'id': 'c587fb15-290d-40e8-a4db-469fd8149fae', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1254, 'output_tokens': 122, 'total_tokens': 1376, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 99}}), ToolMessage(content='{"query": "Spotify potential red flags as business partner", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://open.spotify.com/episode/4iyghhTWsV3LFvin12xKg8", "title": "Legal partnership red flags in business - explained by a lawyer", "content": "... Spotify. Let\'s understand the potential red flags around legal partnerships in businessHave you ever found yourself caught up in the", "score": 0.6685563, "raw_content": null}, {"url": "https://insightxintel.com/barcelona-spotify-sports-sponsorship-deal/", "title": "Red flags from Barcelona FC\'s deal with Spotify - InsightX", "content": "Barcelona FC has announced a £236m sponsorship deal with Spotify, which includes the men\'s and women\'s teams bearing the logo on their shirts.", "score": 0.52802294, "raw_content": null}, {"url": "https://jesterhoax.medium.com/the-spotify-pitch-fc7e75e815c7", "title": "The Spotify Pitch. How we used a comic-book and a big idea…", "content": "They were not publicly listed. There was no tenders. (this was usually the biggest red flag); There were no non-disclosure agreements. There", "score": 0.4877716, "raw_content": null}, {"url": "https://podcasters.spotify.com/pod/show/laconyamurray/episodes/164--Dont-Sign-That-Yet--Important-Contract-Red-Flags-Every-Business-Owner-Should-Recognize-e35pur2", "title": "Own Your Genius with LaConya Murray - Spotify for Creators", "content": "You\'ll learn how to spot red flags, negotiate terms, and protect your business like a CEO. [Register now – it\'s free to attend live. Replay available for to all", "score": 0.4296453, "raw_content": null}, {"url": "https://daimoon.media/knowledge/music-promotion-services-to-avoid/", "title": "Music Promotion Services and red flags that you should absolutely ...", "content": "2. Social Media Promotion Scams · Followers – Who are they? Do they have profile pictures? · Post engagement – Do they have 1m followers but 100 likes on their", "score": 0.32341266, "raw_content": null}], "response_time": 1.23, "request_id": "20a3d550-add0-4d64-bf09-2a956eca883e"}', name='tavily_search', id='0b615474-e372-426a-8c26-0851f294cc0d', tool_call_id='c587fb15-290d-40e8-a4db-469fd8149fae'), AIMessage(content='When considering Spotify as a potential business partner, some general "red flags" to be aware of, based on the search results, could include:\n\n*   **Lack of Public Listing/Tenders:** One article mentioned a past situation where Spotify was not publicly listed and there were no tenders, which was considered a "biggest red flag." This could imply a lack of transparency in certain business dealings.\n*   **Absence of NDAs:** The same article also highlighted the absence of non-disclosure agreements in a particular pitch, which could be a concern for protecting sensitive information.\n*   **Sponsorship Deal Specifics:** While not a direct "red flag" about Spotify\'s general business practices, the article discussing the Barcelona FC sponsorship deal mentioned "red flags" from *that specific deal*. This suggests that the terms and transparency of individual agreements with Spotify should be scrutinized carefully.\n\nIt\'s important to remember that these are past observations or specific instances, and a comprehensive assessment would require due diligence on the current business practices and the specific type of partnership you are considering.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--6a443009-576c-445b-9a5e-97960dd7e668-0', usage_metadata={'input_tokens': 1930, 'output_tokens': 218, 'total_tokens': 2148, 'input_token_details': {'cache_read': 0}})]}
